In [2]:
import math
from IPython.display import display, Math
import json
from pydantic import BaseModel, Field
from collections import Counter
from fractions import Fraction
from include.generators.factorize_smarter import FactorisedNumber, FactorizeSmarter
import logging
fs = FactorizeSmarter()


In [3]:



class Polynomial(BaseModel):
    variable: str = Field(default="x", description="Variable of the polynomial")
    coefficents: list[Fraction] = Field(default=[], description="Exponents of the polynomial, where the first is the lowest order term, i.e x^0 (1)")
    coefficients_neg_exp: list[Fraction] = Field(default=[], description="Negative exponents of the polynomial, where the first is the lowest order term, i.e x^-1")
    def frac_as_str(self, fraction: Fraction):
        if  fraction.is_integer():
            return f"{int(fraction)}"
        else:
            return "\\frac{%g}{%g}" % (fraction.numerator, fraction.denominator)
    def latex_dump(self):

        terms = []

        for exponent in range(0, len(self.coefficents)):

            if self.coefficents[exponent] == 0 & exponent != 0:
                continue # There is no need to print the term, as it is 0, as long as it is not both coefficient and exponent
            elif self.coefficents[exponent] == 1 & exponent != 0:
                coefficient = "" # If exponent is not 0, and the coefficient is 1, we will not print it
            elif self.coefficents[exponent] < 0:
                coefficient = f"- {self.frac_as_str(abs(self.coefficents[exponent]))}" # Prettier way of writing -1
            else:
                coefficient = f"+ {self.frac_as_str(self.coefficents[exponent])}"


            if exponent == 0:
                var_exponent = " " # x^0 is equal to 1, so we don't print it as 1 * y is y
            elif exponent == 1:
                var_exponent = f"{self.variable} "
            else:
                var_exponent = f"{self.variable}^{exponent} "

            term = f"{coefficient}{var_exponent}"
            print(term)
            terms.insert(0, term)
        response = "".join(terms)
        if response[0] == "+":
            return response[1:] # We don't want the first +'
        return "".join(terms)
        print(terms)

display(Math(Polynomial(coefficents=[-0.5,-2,3]).latex_dump()))


- \frac{1}{2} 
- 2x 
+ 3x^2 


<IPython.core.display.Math object>

In [4]:
logging.basicConfig(level=logging.DEBUG)



def find_sum_product_pair(sum: int| Fraction | FactorisedNumber, product: int | Fraction| FactorisedNumber ) -> tuple[Fraction, Fraction]:
    # First lets make certain we have a factorised number
    if isinstance(sum, Fraction) | isinstance(product, int):
        product = fs.factoize(product)
    if isinstance(product, Fraction) | isinstance(sum, int):
        sum = fs.factoize(sum)


    logging.debug(f"My factors are")
    logging.debug(f"Sum: {sum.value} factors {sum.factors}")
    logging.debug(f"Product: {product.value} factors {product.factors}")


    # Identify if both have the same sign (positive or negative)
    if  product.value > 0 :
        logging.debug("As product is positive, the terms must be added and both terms must have same number")
        target = abs(sum.value)
        logging.debug(f"Target {target}")
        for split_by in range(0, len(product.factors)):
            term1 = math.prod(product.factors[:split_by])
            term2 = math.prod(product.factors[split_by:])
            logging.debug(f"Split by position {split_by}, Term1 = {term1} and term2 = {term2} (sum {term1 + term2} product {term1 * term2})")
            if term1 + term2 == target:
                if sum.value < 0:
                    logging.debug("Sum is negative, so both sums must be negative")
                    term1 = term1 * -1
                    term2 = term2 * -1
                logging.debug(f"Found a pair {term1} + {term2} = {sum.value} and {term1} * {term2} = {product.value}")
                return Fraction(term1), Fraction(term2)
        return Fraction(0), Fraction(0)
    elif product.value < 0:
        logging.debug("Product is negative, so we will need to subtrack one term from the other, so we +term1 -term2")
        target = sum.value * -1
        logging.debug(f"Target {target}")
        for split_by in range(0, len(product.factors)):
            term1 = math.prod(product.factors[:split_by])
            term2 = math.prod(product.factors[split_by:])
            logging.debug(f"Split by position {split_by}, Term1 = {term1} and term2 = {term2} (sum {term1 + term2} product {term1 * term2})")
            if term1 - term2 == target:
                logging.debug(f"Found a pair {term1} - {term2} = {sum.value} and {term1} * {term2} = {product.value}")
                return Fraction(term1), Fraction(-term2)
        return Fraction(0), Fraction(0)
    if sum.value < 0 and product.value < 0:
        pass


    #print(json.dumps(product.model_dump(), indent=4))

resp = find_sum_product_pair(63, 180)
print(resp)

DEBUG:root:My factors are
DEBUG:root:Sum: 63 factors [3, 3, 7]
DEBUG:root:Product: 180 factors [2, 2, 3, 3, 5]
DEBUG:root:As product is positive, the terms must be added and both terms must have same number
DEBUG:root:Target 63
DEBUG:root:Split by position 0, Term1 = 1 and term2 = 180 (sum 181 product 180)
DEBUG:root:Split by position 1, Term1 = 2 and term2 = 90 (sum 92 product 180)
DEBUG:root:Split by position 2, Term1 = 4 and term2 = 45 (sum 49 product 180)
DEBUG:root:Split by position 3, Term1 = 12 and term2 = 15 (sum 27 product 180)
DEBUG:root:Split by position 4, Term1 = 36 and term2 = 5 (sum 41 product 180)


(Fraction(0, 1), Fraction(0, 1))


In [12]:
def all_factors(product: int| Fraction | FactorisedNumber):
    # First lets make certain we have a factorised number
    if isinstance(sum, Fraction) | isinstance(product, int):
        product = fs.factoize(product)

    values = product.factors
    logging.debug(f"Complex number: {product.complex_number}")
    logging.debug(f"{values}, {len(values)}")
    if not product.complex_number:
        logging.debug("Product is a prime number, so it only has one factor (it self)")
        return 1, values[0]
    if len(values) == 2:
        logging.debug("Product only has two factors")
        return values[0], values[1]

    # As we only have to take just under half of all numbers
    print(len(values)//2)
    for factor_len in range(1, len(values)//2 + 1):
        logging.debug(f"Trying to find factors of length {factor_len}")

all_factors(9)
all_factors(11)
all_factors(18000)


DEBUG:root:Complex number: True
DEBUG:root:[3, 3], 2
DEBUG:root:Product only has two factors
DEBUG:root:Complex number: False
DEBUG:root:[11], 1
DEBUG:root:Product is a prime number, so it only has one factor (it self)
DEBUG:root:Complex number: True
DEBUG:root:[2, 2, 2, 2, 3, 3, 5, 5, 5], 9
DEBUG:root:Trying to find factors of length 1
DEBUG:root:Trying to find factors of length 2
DEBUG:root:Trying to find factors of length 3
DEBUG:root:Trying to find factors of length 4


4
